For Analysis of Calculations
Run this script first! 
It will produce folders named \<iso-date\>_analysis_output_vx.py
The calculations are the same as in phenometrics.py. The output of this script is the used in the script 
phenometrics_pixel_to_graph.ipynb.



In [89]:
import os
import numpy as np
import pandas as pd
import concurrent.futures
import rasterio
from scipy.interpolate import splrep, BSpline
from sklearn.linear_model import LinearRegression
from datetime import datetime


In [90]:
# Constants
SMOOTH = 0.05
THRESHOLD_ = 90 # can be set to None
THRESH_END = 250


# select with these three constants which vertical pixelstack will be analized. 
# Do not select too many pixels!!
LOAD_MIN_COL = 964
LOAD_MAX_COL = LOAD_MIN_COL + 1
LOAD_MIN_ROW = 628
LOAD_MAX_ROW = 647



In [91]:

def load_ndvi_stack(directory, filter_threshold=THRESHOLD_):
    """
    Load NDVI stack from a directory containing NDVI images.
    The threshold is set from the constants if not set manually to an int between 0 and 365.
    A two dimensional mask is returned that is true for all pixels that hold at least one value 
    """
    print('Loading NDVI stack from', directory)
    # TODO: shouldn't the first dataset below the threshold be taken into consideration?

    file_paths = sorted([os.path.join(directory, fp) for fp in os.listdir(directory) if fp.endswith('.tif')])
    doys = np.array([int(os.path.basename(fp).split('_')[-2].split('.')[0]) for fp in file_paths], dtype=np.int64)
    
    if filter_threshold is not None:
        file_paths = [fp for fp in file_paths if int(os.path.basename(fp).split('_')[-2].split('.')[0]) >= filter_threshold]
        doys = np.array([int(os.path.basename(fp).split('_')[-2].split('.')[0]) for fp in file_paths], dtype=np.int64)
          
    ndvi_stack = []
    mask_stack = []
       
    for fp in file_paths:
        with rasterio.open(fp) as dataset:
            ndvi_data = dataset.read(1).astype(np.float64)
            ndvi_stack.append(ndvi_data[LOAD_MIN_ROW:LOAD_MAX_ROW, LOAD_MIN_COL:LOAD_MAX_COL])
            # ndvi_stack.append(ndvi_data)
            
            mask_stack.append(np.isnan(ndvi_data[LOAD_MIN_ROW:LOAD_MAX_ROW, LOAD_MIN_COL:LOAD_MAX_COL]))
    with rasterio.open(file_paths[0]) as dataset:
        meta = dataset.meta

    ndvi_stack = np.array(ndvi_stack)
    mask_stack = np.array(mask_stack)

    valid_image_area_mask = ~np.any(mask_stack, axis=0)
    if len(doys) < 3:
        raise ValueError(f'Not enough images in the stack (less than 3!) in directory {directory} with threshold {filter_threshold}')
    
    return ndvi_stack, doys, valid_image_area_mask, meta


In [92]:
def find_max(doys, ndvi_values, THRESH_END):
    mask = doys <= THRESH_END
    filtered_doys = doys[mask]
    filtered_ndvi_values = ndvi_values[mask]
    max_index = np.argmax(filtered_ndvi_values)
    return filtered_doys[max_index], filtered_ndvi_values[max_index]

In [147]:
def filter_to_range_between_minima(fine_doys, b_spline, max_ndvi_doy):
    mask_sos = fine_doys < max_ndvi_doy
    mask_eos = fine_doys > max_ndvi_doy
    
    filtered_doys_sos = fine_doys[mask_sos]
    filtered_b_spline_sos = b_spline[mask_sos]
    
    filtered_doys_eos = fine_doys[mask_eos]
    filtered_b_spline_eos = b_spline[mask_eos]
    
    # indices of minimum NDVI values on both sides of max_ndvi_doy
    index_sos = np.argmin(filtered_b_spline_sos)
    index_eos = np.argmin(filtered_b_spline_eos)
    
    # Convert these indices back to the original indices
    original_index_sos = np.where(fine_doys == filtered_doys_sos[index_sos])[0][0]
    original_index_eos = np.where(fine_doys == filtered_doys_eos[index_eos])[0][0]
    
    # subsets of fine_doys and b_spline between the two NDVI minima
    b_spline_new = b_spline[original_index_sos:(original_index_eos+1)]
    fine_doys_new = fine_doys[original_index_sos:(original_index_eos+1)]
    
    
    return fine_doys_new, b_spline_new

In [93]:
def remove_nan(arr1, arr2):
    mask = ~np.isnan(arr2)
    return arr1[mask], arr2[mask]

In [94]:
def fit_linear_regression(fine_doys, b_spline, max_ndvi_doy):
    left_indices = fine_doys < max_ndvi_doy
    right_indices = fine_doys > max_ndvi_doy

    left_model = LinearRegression().fit(fine_doys[left_indices].reshape(-1, 1), b_spline[left_indices]) if np.any(left_indices) else None
    right_model = LinearRegression().fit(fine_doys[right_indices].reshape(-1, 1), b_spline[right_indices]) if np.any(right_indices) else None
    
    return left_model, right_model

In [120]:
def calculate_base(left_model, right_model):
    
    if left_model is not None and right_model is not None:
        return (left_model.coef_[0] + right_model.coef_[0]) / 2.0
    else:
        return None

In [96]:
def calculate_relative_amplitude(ndvi_values):
    return np.percentile(ndvi_values, 90) - np.percentile(ndvi_values, 10)

In [97]:
def convert_to_serializable(obj):
    if obj is np.nan:
        return None
    if isinstance(obj, np.generic):
        return obj.item()
    return obj

In [123]:
def calculate_sos_eos(fine_doys, b_spline, base, amplitude, max_ndvi_doy, max_ndvi_value, overall_relative_amplitude):
    """
    Calculations are split into sos - start of season and eos - end of season.
    Those timeranges are defined by the maximum NDVI value in the seaseon.
    """
    sos_mask = fine_doys < max_ndvi_doy
    eos_mask = fine_doys > max_ndvi_doy
    sos_doys = fine_doys[sos_mask]
    eos_doys = fine_doys[eos_mask]
    sos_ndvi_values = b_spline[sos_mask]
    eos_ndvi_values = b_spline[eos_mask]

    seasonal_amplitude = base + 0.25 * amplitude 
    len_sos_ndvi_values = True
    len_eos_ndvi_values = True
    
    # SoS calculations
    if len(sos_ndvi_values) > 0:
        sos_min_value = np.min(sos_ndvi_values)
        threshold_sos = sos_min_value + 0.1 * (max_ndvi_value - sos_min_value)
        #TODO Genaugenommen müsste hier der np.abs wegfallen, weil ABOVE thrs
        # JETZT ERSETZT DURCH first_of_slope10_sos2
        # first_of_slope10_sos = sos_doys[np.argmin(np.abs(sos_ndvi_values - threshold_sos))]
        
        sos_diff = sos_ndvi_values - threshold_sos
        positive_sos_diff = sos_diff[sos_diff > 0]
        if len(positive_sos_diff) > 0:
        # TODO delete this or first_of-slope10_sos
            first_of_slope10_sos2_doy = sos_doys[sos_diff > 0][np.argmin(sos_diff[sos_diff > 0])]
        else:
            first_of_slope10_sos2_doy = np.nan
            print(f'Pixel first_of_slope10_sos2_doy is nan')
        
        median_ndvi_sos = np.median(sos_ndvi_values)
        sos_median_diff = sos_ndvi_values - median_ndvi_sos
        median_of_slope_sos_doy = sos_doys[np.argmin(np.abs(sos_ndvi_values - median_ndvi_sos))] 
         
        seasonal_amplitude_doy_sos = sos_doys[np.argmin(np.abs(sos_ndvi_values - seasonal_amplitude))]
        
        relative_amplitude_sos_old = calculate_relative_amplitude(sos_ndvi_values)
        relative_amplitude_doy_sos_old_idx = np.argmin(np.abs(sos_ndvi_values - relative_amplitude_sos_old))
        relative_amplitude_doy_sos_old = sos_doys[relative_amplitude_doy_sos_old_idx]
        if relative_amplitude_doy_sos_old_idx == 0:
            relative_amplitude_doy_sos_old = np.nan


        # relative to the overall amplitude
        
        dists_from_relative_amplitude = abs(sos_ndvi_values - overall_relative_amplitude)
        idx_relative_amplitude = np.argmin(dists_from_relative_amplitude)
        if idx_relative_amplitude > 0:
            relative_amplitude_doy_sos = sos_doys[idx_relative_amplitude]
        else:
            relative_amplitude_doy_sos = np.nan

    else:
        len_sos_ndvi_values = False
        # first_of_slope10_sos = np.nan
        first_of_slope10_sos2_doy = np.nan
        median_of_slope_sos_doy = np.nan
        seasonal_amplitude_doy_sos = np.nan
        relative_amplitude_doy_sos = np.nan
        relative_amplitude_doy_sos_old = np.nan

    # EoS calculations
    if len(eos_ndvi_values) > 0:
        eos_min_value = np.min(eos_ndvi_values)
        threshold_eos = eos_min_value + 0.1 * (max_ndvi_value - eos_min_value)
        # Siehe oben!
        # first_of_slope10_eos = eos_doys[np.argmin(np.abs(eos_ndvi_values - threshold_eos))]

        eos_diff = eos_ndvi_values - threshold_eos
        positive_eos_diff = eos_diff[eos_diff < 0]
        if len(positive_eos_diff) > 0:
        # TODO delete this or first_of-slope10_sos
            first_of_slope10_eos2_doy = eos_doys[eos_diff < 0][np.argmax(eos_diff[eos_diff < 0])]
        else:
            first_of_slope10_eos2_doy = np.nan
            print(f'Pixel first_of_slope10_eos2 is nan')
        
        median_ndvi_eos = np.median(eos_ndvi_values)
        eos_median_diff = eos_ndvi_values - median_ndvi_eos
        
        median_of_slope_eos_doy = eos_doys[np.argmin(np.abs(eos_ndvi_values - median_ndvi_eos))]
        
        
        seasonal_amplitude_doy_eos = eos_doys[np.argmin(np.abs(eos_ndvi_values - seasonal_amplitude))]
        
        # old relative amplitude
        relative_amplitude_eos_old = calculate_relative_amplitude(eos_ndvi_values)
        relative_amplitude_eos_old_idx = np.argmin(np.abs(eos_ndvi_values - relative_amplitude_eos_old))
        relative_amplitude_doy_eos_old = eos_doys[relative_amplitude_eos_old_idx]
        if relative_amplitude_eos_old_idx == 0:
            relative_amplitude_doy_eos_old = np.nan

        # relative to the overall amplitude
        dists_from_relative_amplitude = abs(eos_ndvi_values - overall_relative_amplitude)  
        idx_relative_amplitude = np.argmin(dists_from_relative_amplitude)
        if idx_relative_amplitude > 0:
            relative_amplitude_doy_eos = eos_doys[idx_relative_amplitude]
        else:
            relative_amplitude_doy_eos = np.nan
    else:
        len_eos_ndvi_values = False
        first_of_slope10_eos2_doy = np.nan
        median_of_slope_eos_doy = np.nan
        seasonal_amplitude_doy_eos = np.nan
        relative_amplitude_doy_eos = np.nan
        relative_amplitude_doy_eos_old = np.nan
    
    return {
        'seasonal_amplitude': convert_to_serializable(seasonal_amplitude) if np.any(seasonal_amplitude) else None,
        'len_sos_ndvi_values': convert_to_serializable(len_sos_ndvi_values) if np.any(len_sos_ndvi_values) else None,
        'len_eos_ndvi_values': convert_to_serializable(len_eos_ndvi_values) if np.any(len_eos_ndvi_values) else None,
        'sos_mask': list(convert_to_serializable(sos_mask)) if np.any(sos_mask) else None,
        'eos_mask': list(convert_to_serializable(eos_mask)) if np.any(eos_mask) else None,
        'sos_doys': list(convert_to_serializable(sos_doys)) if np.any(sos_doys) else None,
        'eos_doys': list(convert_to_serializable(eos_doys)) if np.any(eos_doys) else None,
        'sos_ndvi_values': list(convert_to_serializable(sos_ndvi_values)) if np.any(sos_ndvi_values) else None,
        'eos_ndvi_values': list(convert_to_serializable(eos_ndvi_values)) if np.any(eos_ndvi_values) else None,
        'sos_threshold': convert_to_serializable(threshold_sos) if np.any(threshold_sos) else None,
        'sos_median_ndvi': convert_to_serializable(median_ndvi_sos) if np.any(median_ndvi_sos) else None,
        'sos_median_diff': list(convert_to_serializable(sos_median_diff)) if np.any(sos_median_diff) else None,
        'sos_first_of_slope_doy': convert_to_serializable(first_of_slope10_sos2_doy) if np.any(first_of_slope10_sos2_doy) else None, 
        'sos_median_of_slope_doy': convert_to_serializable(median_of_slope_sos_doy) if np.any(median_of_slope_sos_doy) else None, 
        'sos_seasonal_amplitude_doy': convert_to_serializable(seasonal_amplitude_doy_sos) if np.any(seasonal_amplitude_doy_sos) else None, 
        'sos_relative_amplitude_doy': convert_to_serializable(relative_amplitude_doy_sos) if np.any(relative_amplitude_doy_sos) else None,
        'sos_relative_amplitude_doy_old': convert_to_serializable(relative_amplitude_doy_sos_old) if np.any(relative_amplitude_doy_sos_old) else None,
        'eos_threshold': convert_to_serializable(threshold_eos) if np.any(threshold_eos) else None,
        'eos_median_ndvi': convert_to_serializable(median_ndvi_eos) if np.any(median_ndvi_eos) else None,
        'eos_median_diff': list(convert_to_serializable(eos_median_diff)) if np.any(eos_median_diff) else None,
        'eos_first_of_slope_doy': convert_to_serializable(first_of_slope10_eos2_doy) if np.any(first_of_slope10_eos2_doy) else None,
        'eos_median_of_slope_doy': convert_to_serializable(median_of_slope_eos_doy) if np.any(median_of_slope_eos_doy) else None,
        'eos_seasonal_amplitude_doy': convert_to_serializable(seasonal_amplitude_doy_eos) if np.any(seasonal_amplitude_doy_eos) else None,
        'eos_relative_amplitude_doy': convert_to_serializable(relative_amplitude_doy_eos) if np.any(relative_amplitude_doy_eos) else None,
        'eos_relative_amplitude_doy_old': convert_to_serializable(relative_amplitude_doy_eos_old) if np.any(relative_amplitude_doy_eos_old) else None,
    }


In [134]:

def process_pixel(row, col, ndvi_values, doys, num_cols):
    """
    1. Nan values in the pixel are removed
    2. Spline function is determined
    3. the overall relative amplitude is determined
    4. The result for one pixel is completed with the values from sos_eos_calculate    
    """
    print('process_pixel ', row, col)

    # if there are invalid values, they are taken out of the ndvi_values and doys
    isnan_mask = np.isnan(ndvi_values)
    if np.any(isnan_mask):
        ndvi_values = ndvi_values[~isnan_mask]
        doys = doys[~isnan_mask]

    # create an array with all doys in the timerange
    min_doy, max_doy = min(doys), max(doys)
    fine_doys = np.linspace(min_doy, max_doy, max_doy - min_doy + 1, dtype=int)

    tck_spline = splrep(doys, ndvi_values, s=SMOOTH)
    b_spline = BSpline(*tck_spline)(fine_doys)
    print('fine_doys 1', fine_doys)
    max_ndvi_doy, max_ndvi_value = find_max(fine_doys, b_spline, THRESH_END)

    #limit the doy interval to the range between the sos and eos minimum
    fine_doys, b_spline = filter_to_range_between_minima(fine_doys, b_spline, max_ndvi_doy)
    
     
    print('fine_doys 2 at ',row, col, ':\n', fine_doys)


    left_model, right_model = fit_linear_regression(fine_doys, b_spline, max_ndvi_doy)
    base = calculate_base(left_model, right_model)
    amplitude = max_ndvi_value - base

    overall_relative_amplitude = calculate_relative_amplitude(ndvi_values)


    sos_eos_dict = calculate_sos_eos(fine_doys,  b_spline, base, amplitude, max_ndvi_doy, max_ndvi_value, overall_relative_amplitude)

    sos_eos_dict.update({'row': LOAD_MIN_ROW + row, 
                        'col': LOAD_MIN_COL + col, 
                        'org_ndvi_values': list(ndvi_values), 
                        'org_ndvi_doys': list(doys),
                        'min_doy': convert_to_serializable(min_doy), 
                        'max_doy': convert_to_serializable(max_doy), 
                        'max_ndvi_doy': convert_to_serializable(max_ndvi_doy), 
                        'max_ndvi_value': convert_to_serializable(max_ndvi_value),
                        'fine_doys': list(fine_doys),
                        'b_spline': list(b_spline),
                        'base': convert_to_serializable(base),
                    'amplitude': convert_to_serializable(amplitude),
                    'overall_relative_amplitude': convert_to_serializable(overall_relative_amplitude),
                    'left_model': left_model.get_params() if left_model is not None else None,
                    'right_model': right_model.get_params() if right_model is not None else None,
                    'left_model_coef': left_model.coef_[0] if left_model is not None else None,
                    'right_model_coef': right_model.coef_[0] if right_model is not None else None,
                    })

    return sos_eos_dict

In [100]:
def save_geotiff(data, meta, file_path):
    
    meta.update(dtype=rasterio.float32, count=1)
    with rasterio.open(file_path, 'w', **meta) as dst:
        dst.write(data.astype(rasterio.float32), 1)


In [125]:
# No Threading etc - 69minutes
def process_stack(input_dir):
    """Main function to process NDVI data."""
    print(f'Start processing {input_dir}')
    start = datetime.now()
    current_date = datetime.now().date().isoformat()

    def make_directory(input_dir, counter=1):
        output_dir = os.path.join(os.path.dirname(os.path.dirname(input_dir)), f'{current_date}_analysis_output_v{counter}')
        if os.path.exists(output_dir):
            return make_directory(input_dir, counter + 1)
        else:
            os.makedirs(output_dir)
        return output_dir

    output_dir = make_directory(input_dir)

   
    ndvi_stack, doys, mask, meta = load_ndvi_stack(input_dir)
    num_rows, num_cols = ndvi_stack[0].shape
    print(input_dir, ndvi_stack[0].shape)
    # Ensure sos_eos_data has enough depth to store all keys from the dictionary
        
    for row in range(num_rows):
        for col in range(num_cols):
            if mask[row, col]:
                try:  
                    sos_eos_dict = process_pixel(row, col, ndvi_stack[:, row, col], doys, num_cols)
                    with open(os.path.join(output_dir, f'S2_{LOAD_MIN_ROW + row}_{LOAD_MIN_COL + col}.py'), 'a') as f:
                        f.write('ndvi_dict = {\n')
                        for key, value in sos_eos_dict.items():
                            f.write(f'    "{key}": {value},\n')   
                        f.write('}\n')
                except Exception as e:
                    print(f'Error processing pixel {row}, {col}: {e}')
                    continue

               

    
    # An output folder is created at the same level as the input_dir. The output folder is named e.g. 2024-11-06_output.
    





In [148]:
process_stack('./S2_clip/')


Start processing ./S2_clip/
Loading NDVI stack from ./S2_clip/
./S2_clip/ (19, 1)
process_pixel  0 0
fine_doys 1 [111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128
 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146
 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164
 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182
 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200
 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218
 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236
 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254
 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272
 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289 290
 291 292 293 294 295 296 297 298 299 300 301 302 303 304 305 306 307 308
 309 310 311 312 313 314 315 316 317 318 319 320 321 322 323 324 325 326
 327 328 32

In [142]:
def filter_to_range_between_minima(fine_doys, b_spline, max_ndvi_doy):
    
    mask_sos = fine_doys < max_ndvi_doy
    mask_eos = fine_doys > max_ndvi_doy
    
    filtered_doys_sos = fine_doys[mask_sos]
    index_sos = np.argmin(b_spline[mask_sos])
    index_eos = np.argmin(b_spline[mask_eos])
    print('b_spline[mask_eos]', b_spline[mask_eos])
    print('b_spline min', np.min(b_spline[mask_sos]))
    b_spline_new = b_spline[index_sos:(index_eos+1)]
    fine_doys_new = fine_doys[index_sos:(index_eos+1)]
    print('index_sos', index_sos, 'SOS doy',fine_doys[index_sos])
    print('index_eos', index_eos, 'Eos doy', fine_doys[index_eos])
    print('fine_doys after', fine_doys_new)
    

    # min_ndvi_doy_sos = filtered_doys_sos[np.argmin(filtered_ndvi_values_sos)]
    
    # filtered_doys_eos = fine_doys[mask_eos]
    
    
    # filtered_ndvi_values_eos = b_spline[mask_eos]
    
    
    # min_ndvi_doy_eos = filtered_doys_eos[np.argmin(filtered_ndvi_values_eos)]
    
    return fine_doys_new, b_spline_new

In [146]:
fine_doys = np.array([100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200])
b_spline = np.array([0.3, 0.2, 0.4, 0.45, 0.5, 0.6, 0.55, 0.5, 0.45, 0.3, 0.35])
max_ndvi_doy = 150

fine_doys_new, b_spline_new = filter_to_range_between_minima(fine_doys, b_spline, max_ndvi_doy)

print('Fine doys new:', fine_doys_new)
print('B spline new:', b_spline_new)


Filtered b_spline eos: [0.55 0.5  0.45 0.3  0.35]
Min b_spline sos: 0.2
Index SOS: 1 SOS doy: 110
Index EOS: 9 EOS doy: 190
Fine doys after: [110 120 130 140 150 160 170 180 190]
Fine doys new: [110 120 130 140 150 160 170 180 190]
B spline new: [0.2  0.4  0.45 0.5  0.6  0.55 0.5  0.45 0.3 ]
